In [ ]:
import pandas as pd
import numpy as np
from binance.client import Client
from datetime import datetime, timedelta, timezone
import os
import pandas_ta as ta
from tqdm import tqdm
import time 
import pytz
import os
# os.system('cls' if os.name == 'nt' else 'clear')
from IPython.display import clear_output
# clear_output(wait=True)

In [16]:
import importlib
import BaseFunctions
importlib.reload(BaseFunctions)
from BaseFunctions import *

import StrategyList
importlib.reload(StrategyList)
from StrategyList import *

In [ ]:
# Load API credentials from environment variables
API_KEY = os.getenv("BinanceAPI_250502")
API_SECRET = os.getenv("BinanceSecret_250502")

In [ ]:
# Initialize Binance client
client = Client(API_KEY, API_SECRET) if API_KEY and API_SECRET else Client()

In [ ]:
# Get market details
market_details = getMarketDetails(client)
# market_details = pd.DataFrame(market_details)
pair_names = market_details['pair'].tolist()
# print(market_details)

# Get balance for a specific asset
getBalance(client, "BTC")

In [ ]:
BackTime = "2025-05-01 00:00:00"
# Convert string to naive datetime object
BackTime = datetime.strptime(BackTime, '%Y-%m-%d %H:%M:%S')
# Make it timezone-aware (UTC)
BackTime = BackTime.replace(tzinfo=timezone.utc)
print(BackTime)

In [ ]:
BackTime = int(BackTime.timestamp() * 1000)  # Convert to milliseconds
Interval = '5m'
current_time = time.time()
pair = 'BTCUSDT'

BuySellFlag = 'Hold'
BuyCounter = 0
SellCounter = 0

In [ ]:
BackCandles = get_candles_data(pair, Interval, 1000, BackTime, client)
# Metric = calculate_metrics(BackCandles, pair)
SCORE = allstrategiesv2(BackCandles)

print(SCORE)

In [ ]:
allcandles = pd.DataFrame()

# while BackTime < current_time * 1000:
while BackTime < time.time() * 1000:

    # Clear the console at the start of each iteration
    clear_output(wait=True)
    
    Lastcandle = pd.DataFrame()

    BackTime = BackTime + (5*60*1000)  # Increment by 5 minutes

    # print(BackTime)
    BackCandles = get_candles_data(pair, Interval, 250, BackTime, client)
    Lastcandle = BackCandles.iloc[[-1]].copy()
    print(Lastcandle)
    # print(Lastcandle.to_frame().T)
    Metric = calculate_metrics(BackCandles, pair)

    selected_columns = ['Per_Volume', 'Latest_MACD_Scenario', 'Latest_MACD_ScenarioCounter', 'MACD_Score', 'DyingCoinFlag_EMA']
    Lastcandle = pd.concat([Lastcandle.reset_index(drop=True), Metric[selected_columns].reset_index(drop=True)], axis=1)

    if Metric['Per_Volume'].iloc[0]>1.75 and Metric['Latest_MACD_Scenario'].iloc[0] == 'Bullish Increasing (Increasing)' and BuySellFlag != 'Buy':
        BuySellFlag = 'Buy'
        Lastcandle['BuySellFlag'] = BuySellFlag
        BuyCounter += 1
        print('Buy Triggered : ', Lastcandle['Close'], " || Counter: ", BuyCounter)
    elif Metric['Per_Volume'].iloc[0]>1.75 and Metric['Latest_MACD_Scenario'].iloc[0] == 'Bearish Decreasing (Decreasing)' and BuySellFlag == 'Buy':
        BuySellFlag = 'Sell'
        Lastcandle['BuySellFlag'] = BuySellFlag
        SellCounter += 1
        print('Sell Triggered : ', Lastcandle['Close'], " || Counter: ", SellCounter)
    else:
        Lastcandle['BuySellFlag'] = 'Hold'
        # print("No Action Triggered (Time: ", BackTime, ")")
    
    allcandles = pd.concat([allcandles, Lastcandle], ignore_index=True)


In [ ]:
testconsolidatedActionData = allcandles.copy()

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class prsStrategy(Strategy):

    def init(self):
        pass

    def next(self):
        if self.data.BuySellFlag == 'Buy':
            self.buy()
        elif self.data.BuySellFlag == 'Sell':
            self.sell()

# BuySellSignal = allcandles.rename(columns={'open': 'Open', 'close': 'Close', 'high': 'High', 'low': 'Low', 'volume': 'Volume'})
bt = Backtest(allcandles, 
              prsStrategy,
              cash=10000,  
              exclusive_orders=True,
              commission = 0.012)

stats = bt.run()
bt.plot()
print(stats)